In [1]:
import gradio as gr
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import (
    MobileNetV2, preprocess_input, decode_predictions
)

# Load pretrained model (hidden)
model = MobileNetV2(weights="imagenet")

# ImageNet class groups
CAT_KEYWORDS = [
    "tabby", "tiger_cat", "persian_cat", "siamese_cat",
    "egyptian_cat"
]

DOG_KEYWORDS = [
    "dog", "puppy", "hound", "terrier", "retriever",
    "shepherd", "bulldog", "poodle", "labrador"
]

def predict_cat_dog(img):
    if img is None:
        return "Please upload an image"

    # Preprocess image
    img = img.convert("RGB")
    img = img.resize((224, 224))
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Predict
    preds = model.predict(img_array)
    decoded = decode_predictions(preds, top=10)[0]

    cat_score = 0
    dog_score = 0

    for _, label, confidence in decoded:
        label = label.lower()
        if any(cat in label for cat in CAT_KEYWORDS):
            cat_score += confidence
        if any(dog in label for dog in DOG_KEYWORDS):
            dog_score += confidence

    # Decision logic
    if cat_score > dog_score and cat_score > 0.2:
        return f"It's a Cat 🐱 (confidence: {cat_score:.2f})"
    elif dog_score > cat_score and dog_score > 0.2:
        return f"It's a Dog 🐶 (confidence: {dog_score:.2f})"
    else:
        return "Unable to confidently classify as Cat or Dog ❌"

# Gradio UI
app = gr.Interface(
    fn=predict_cat_dog,
    inputs=gr.Image(type="pil", label="Upload Image"),
    outputs=gr.Textbox(label="Prediction"),
    title="Accurate Cat vs Dog Classifier",
    description="Upload an image. The system will accurately detect whether it is a Cat or a Dog."
)

app.launch()

ModuleNotFoundError: No module named 'gradio'